# MTAOS handling of rejected commands #

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2193 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2193)
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by E. Dennihy 20210928

***
Load all the needed libraries.
Get the remotes ready
Code in the notebook including section: "Check the summary state of each CSC".

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

***

Ready M1M3:
Raise mirror, turn on FB, clear forces

Need to have M1M3 LUT use its inclinometer.

***

Ready M2:
Turn on FB, clear forces

Need to have M2 LUT use its inclinometer

***

Get camera hexapod ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position


***

Get M2 hexapod ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

***

clear all corrections using cmd_resetCorrection

In [ ]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

***
Add 1um of z7 to the system via OFC, issue the corrections.

Compare the corrections sent vs forces and position changes applied. This is currently done in a separate notebook or on Chronograf. 

In [ ]:
wavefront_errors = np.zeros(19)

In [ ]:
wavefront_errors[3]=1.0

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Make plots using telemetry from each component to verify the changes in the DOFs. This step does not currently involve running any commands in this notebook. This step must be verified using a separate noteboook. 

***
Put M2 hexapod in DISABLED state (so that we can test command rejection).

In [ ]:
await mtcs.set_state(salobj.State.DISABLED, components=["mthexapod_2"])

***
Add 1um of z7 to the system via OFC. Expect m2 hexapod corrections are rejected, and all other corrections applied, then undone.

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Re-enable M2 hexapod
Make it ready for AOS

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mthexapod_2"])

***
Re-issue the correction.

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Reject the latest corrections.

In [ ]:
await mtcs.rem.mtaos.cmd_rejectCorrection.start()

***
Add 2um of z7 via OFC

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

In [ ]:
wavefront_errors[3] = 2.0

In [ ]:
wavefront_errors

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Wrap up. Put each component to the following states:
mtaos --> standby
m1m3 --> standby
m2 --> standby
camera hex --> standby
m2 hex --> standby

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()